In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

# Load Trained Model

In [ ]:
import torch
model_path = './no_monocyte/model_fold_0'
net = torch.load(model_path)
net.to("cpu")
net.eval()

In [ ]:
PATHS = [r"..\data\processed\marrow.pickle", 
         r"..\data\processed\monocyte.pickle",
         r"..\data\processed\alveolar.pickle"]
         
NUM_WORKERS = 2
BATCH_SIZE = 6

In [ ]:
import torchvision.transforms as transforms
from utils import *
transforms = transforms.Compose([
    standardize_input()
    ])

In [ ]:
from torch.utils.data import DataLoader
import pickle
from MacDataset import MacDataset
raw_images = []
raw_labels = []
for i, path in enumerate(PATHS):
    path_data = pickle.load(open(path, "rb"))
    path_data["labels"][:] = i
    raw_images.append(path_data["images"])
    raw_labels.append(path_data["labels"])

images = np.vstack(raw_images)[:,[0],:,:]
labels = np.hstack(raw_labels)

trainset = MacDataset(images, labels, 
                            transform=transforms)
train_sampler = equal_classes_sampler(trainset.labels)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, sampler=train_sampler,
                        shuffle=False, num_workers=0)

In [ ]:
import torchvision
def imshow(img, transpose = True):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize = (11,4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))

l_names = ["marrow", "monocyte", "alveolar"]
print('Ground Truth : ', ' '.join('%10s' % l_names[int(labels[j].item())] 
                            for j in range(BATCH_SIZE)))

pred_names = ["alveolar", "marrow"]
outputs = net(images.float())
predicted = torch.argmax(outputs,1).to(torch.double)
probability = torch.softmax(outputs, 1).detach().numpy()

print('Predicted    : ', ' '.join('%10s' % pred_names[int(predicted[j].item())] 
                            for j in range(BATCH_SIZE)))
print('Alveolar prob: ', ' '.join('%10.5f' % probability[j][0].item()
                            for j in range(BATCH_SIZE)))

print('Marrow prob  : ', ' '.join('%10.5f' % probability[j][1].item()
                            for j in range(BATCH_SIZE)))                              